In [ ]:
%%pyspark project.spark.compatibility
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, length, when
import re


df = spark.read.option("header", "true").csv("rawdata_path")

Executing for connection type: SPARK_GLUE, connection name: project.spark.compatibility
Creating Glue session...
Create session for connection: project.spark.compatibility


'Session ajv487omhsvq1c-db08230b-c7ad-43d9-8683-ab370f0f6ff7 has been created.'

<sagemaker_studio_dataengineering_sessions.sagemaker_base_session_manager.common.debugging_utils.SessionInfoTableDisplay object>

Session created for connection: project.spark.compatibility.

Connection: project.spark.compatibility | Run start time: 2025-12-09 06:11:22.590712 | Run duration : 0:01:53.846085s.


In [4]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df.show()

+--------------------+---------------------------------+----------------------------------+---------+---------+
|                  pk|                            title|                          contents|    label|frequency|
+--------------------+---------------------------------+----------------------------------+---------+---------+
|20210202-233-1100...| [제2차 경기도 재난기본소득 신...|  (광고)수협신용카드/체크카드 ◆...|*********|    85250|
|20191016-84-10709...|Sh수협은행과 카카오톡채널 친구...|    (광고) Sh수협은행 공식 카카...|        Y|    39356|
|20190711-233-1189...|  Sh수협 장기카드대출(카드론) ...|   (광고) 신청 당일 바로 지급되...|        Y|    33751|
|20190619-233-1189...|  Sh수협 장기카드대출(카드론) ...|    (광고) Sh수협 장기카드대출(...|        Y|    31087|
|20191217-84-10709...|  (광고) Sh수협은행과 카카오톡...|   (광고) Sh수협은행 헤이프렌즈...|        Y|    27721|
|20191016-84-10709...|Sh수협은행과 카카오톡채널 친구...|   (광고)Sh수협은행 공식 카카오...|        Y|    20663|
|20191217-84-10709...|  (광고) Sh수협은행과 카카오톡...|   (광고)Sh수협은행 헤이프렌즈 ...|        Y|    19469|
|20211214-808-1100...|            (광고)

In [5]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment

def clean_text_combined(text):
    if text is None:
        return ""
    
    try:
        # HTML 태그와 엔티티 처리
        text = re.sub(r'<br>', ' ', text)
        text = re.sub(r'</br>', ' ', text)
        text = re.sub(r'<br\s*/?>', ' ', text)
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'<[^&]*>', '', text)
        text = re.sub(r'&[a-zA-Z0-9#]+;', '', text)
        
        # URL, 전화번호, 괄호 내용 제거
        text = re.sub(r'http[s]?://\S+', '', text)
        text = re.sub(r'\d{4}-\d{4}', '', text)
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub(r'^\(광고\)', '', text)
        
        # 특정 키워드 포함 줄 제거
        lines = text.split('\n')
        filtered_lines = []
        for line in lines:
            if not any(keyword in line for keyword in ['무료수신거부', '준법', '광고']):
                filtered_lines.append(line)
        
        text = '\n'.join(filtered_lines)
        
        # 특수문자 제거
        special_chars = ['※', '◆', '■', '▶', '★', '☞', '☎', 'br', 'brbr']
        for char in special_chars:
            text = text.replace(char, '')
        
        # 연속된 공백과 줄바꿈 정리
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    except:
        return text


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:13:24.199716 | Run duration : 0:00:03.424462s.


In [6]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
clean_text_combined_udf = udf(clean_text_combined, StringType())
df_final = df.withColumn("contents_final", clean_text_combined_udf(col("contents"))) \
            .filter(length(col("contents_final")) > 30)


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:13:27.633078 | Run duration : 0:00:03.400139s.


In [7]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two=df_final.select('title', 'contents_final', 'label', 'frequency')


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:13:31.040886 | Run duration : 0:00:03.276484s.


In [8]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two.show()

+-------------------------------------+-----------------------------------+---------------------------------+---------------------------------+
|                                title|                     contents_final|                            label|                        frequency|
+-------------------------------------+-----------------------------------+---------------------------------+---------------------------------+
|                      (광고) 부임임사| 부임임사 안녕하세요 반갑습니다....|                                N|                             3820|
|        (광고)수협은행 광교신도시지점|수협은행 광교신도시지점 안녕하세...|                                Y|                             3634|
|                                 NULL|설연휴~가족과행복한시간보내시고 ...|                                N|                             3354|
|                       우수고객님께^^|      "[제1금융기관] sh수협은행 ...|   지금! “수협은행 수유동지점”...|   매주 변동 가능성 있습니다. ...|
|     [Sh수협은행] 라온이 인증샷이벤트|    [Sh수협은행] 라온이인형과 함...|                                Y|       

In [9]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered = df_cleaned_two.filter(col("label").isin(["Y", "N"]))


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:13:48.803096 | Run duration : 0:00:03.329639s.


In [10]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered.count()


23
Connection: project.spark.compatibility | Run start time: 2025-12-09 06:13:52.141253 | Run duration : 0:00:08.721716s.


In [11]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final = df_filtered.filter(col("label").isin(["Y", "N"])) \
    .withColumn("label", when(col("label") == "Y", 1).otherwise(0)) \
    .withColumn("frequency", col("frequency").cast("int"))


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:14:00.870850 | Run duration : 0:00:04.231706s.


In [12]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final.show()

+-------------------------------------+-----------------------------------+-----+---------+
|                                title|                     contents_final|label|frequency|
+-------------------------------------+-----------------------------------+-----+---------+
|                      (광고) 부임임사| 부임임사 안녕하세요 반갑습니다....|    0|     3820|
|        (광고)수협은행 광교신도시지점|수협은행 광교신도시지점 안녕하세...|    1|     3634|
|                                 NULL|설연휴~가족과행복한시간보내시고 ...|    0|     3354|
|     [Sh수협은행] 라온이 인증샷이벤트|    [Sh수협은행] 라온이인형과 함...|    1|      570|
|  은영노인요양시설 신용카드 발급 안내| 안녕하세요~ 수협은행 철산역지점...|    1|       51|
|   수협은행 또잇!간편대출상품 안내...|    [수협은행]디지털 브로** 고객...|    1|       49|
|            [수협은행 부경대금융센터]|  [수협은행 부경대금융센터][수협...|    0|       24|
|             ♥수협은행재예치상품추천♥| [수협은행]디지털브로슈어 추천은...|    1|       22|
|                                    1|     "수협은행""평생적용이율 2.6...|    1|       20|
|   [광고]고양삼송우미라피아노 고객님.|  고객님 발급하신 수협 신용카드 ...|    0|       17|
|                           [수협은행

In [13]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
# label별로 분리
df_label_0 = df_filtered_final.filter(col("label") == 0)
df_label_1 = df_filtered_final.filter(col("label") == 1)

# 각각 split
train_0, test_0 = df_label_0.randomSplit([0.7, 0.3], seed=42)
train_1, test_1 = df_label_1.randomSplit([0.7, 0.3], seed=42)

# 합치기
train_df = train_0.union(train_1).orderBy(rand(seed=42))
test_df = test_0.union(test_1).orderBy(rand(seed=42))


Connection: project.spark.compatibility | Run start time: 2025-12-09 06:14:13.880633 | Run duration : 0:00:03.419250s.


In [14]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.show()

+----------------------------------+-----------------------------------+-----+---------+
|                             title|                     contents_final|label|frequency|
+----------------------------------+-----------------------------------+-----+---------+
| (광고)수협은행 헤이정기예금 연...| [수협은행]디지털브로슈어 추천은...|    1|       64|
|           (광고)수협은행 영도지점| 수협은행 영도지점 함께하면 이겨...|    1|      145|
|   (광고)[수협은행 압구정금융센터]|  [수협은행 압구정금융센터] 안녕...|    0|       65|
|수협은행 또잇!간편대출상품 안내...|    [수협은행]디지털 브로** 고객...|    1|       49|
|                              NULL| 행복한 한가위명절 보내시기 바랍...|    0|       62|
|         [수협은행 부경대금융센터]|  [수협은행 부경대금융센터][수협...|    0|       24|
|                              NULL|설연휴~가족과행복한시간보내시고 ...|    0|     3354|
|   (광고)수협은행7월 정기예금 특판|  수협은행7월 정기예금 특판 수협...|    1|      443|
|     (광고)수협은행 광교신도시지점|수협은행 광교신도시지점 안녕하세...|    1|     3634|
|   (광고)◆ 수협카드 국민지원금 ...| 수협카드 국민지원금 신청 함께하...|    0|      103|
|          ♥수협은행재예치상품추천♥| [수협은행]디지털브로슈어 추천은...|    1|       22|
|   (광고)[무]Sh

In [15]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.show()

+-------------------------------------+-----------------------------------+-----+---------+
|                                title|                     contents_final|label|frequency|
+-------------------------------------+-----------------------------------+-----+---------+
|     [Sh수협은행] 라온이 인증샷이벤트|    [Sh수협은행] 라온이인형과 함...|    1|      570|
|          Sh수협은행 예적금 상품 안내|  [수협은행] 디지털브로슈어 추천...|    1|       14|
|   [광고]고양삼송우미라피아노 고객님.|  고객님 발급하신 수협 신용카드 ...|    0|       17|
|                    (광고) Sh수협은행|  Sh수협은행 수협은행 디지털명함...|    0|       14|
|안녕하세요 수협은행인천논현지점입니다|굿모닝요양원 종사자분들에게 적금...|    1|        9|
|  (광고)수협카드 국민지원금 신청 안내|  수협카드 국민지원금 신청 안내 ...|    1|      289|
|                  (광고)수협은행 안내|  수협은행 안내 당행을 이용해 주...|    1|        9|
|  은영노인요양시설 신용카드 발급 안내| 안녕하세요~ 수협은행 철산역지점...|    1|       51|
+-------------------------------------+-----------------------------------+-----+---------+

Connection: project.spark.compatibility | Run start time: 2025-12-09 06:14:26.038652 | Run duration : 

In [ ]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.write.option("header",True) \
 .csv("train_path")



Connection: project.spark.compatibility | Run start time: 2025-12-09 06:14:46.632878 | Run duration : 0:00:11.045613s.


In [ ]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.write.option("header",True) \
 .csv("test_path")



Connection: project.spark.compatibility | Run start time: 2025-12-09 06:14:57.683970 | Run duration : 0:00:09.006883s.


In [ ]:
%%pyspark default.spark
# Enter your code at the start of this line to replace this comment